### 1. 데이터 불러오기 및 전처리

In [ ]:
import numpy as np
import joblib
import json
import pandas as pd
import pymysql
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate, KFold, train_test_split

# 그래프 관련 패키지
import seaborn as sns
import matplotlib.pyplot as plt

plt.rc('font', family = 'NANUMBARUNGOTHIC')
print(plt.rcParams['font.family'])

%matplotlib inline

['NANUMBARUNGOTHIC']


In [ ]:
# 입력값
advertiser_id = '13' # 광고주: 에스티유니타스(13) or None
site_id =  '85' # 사이트: 수험1팀_공단기(85) or None
budget = 100000000 # 예산
kpi = 'click' # KPI(click, cpa, roas)

In [ ]:
filePath = '/home/anaconda3/da/data/media mix_data9/'
result_data = []
output = {}

In [ ]:
# DB 접속(정보 생략)
conn = pymysql.connect()

In [ ]:
# 요약 테이블 불러오기
summary_query = "SELECT * FROM data9_search_summary"
summary_df = pd.read_sql(summary_query, conn)

In [ ]:
# 캠페인 구분 변수 생성
summary_df = summary_df.astype({'statYear': str, 'statMonth': str})
summary_df['campaign'] = summary_df['advertiser_name'] + "_" + summary_df['site_name'] + "_" + summary_df['statYear'] + "_" + summary_df['statMonth']
summary_df['mediaDevice'] = summary_df['media_code'] + "_" + summary_df['deviceType'] ###

# 캠페인 기준 Summary(사이트/월 기준)
summary_df_fin = summary_df.groupby(['advertiser_id', 'advertiser_name', 'site_id', 'site_name', 'statYear', 'statMonth']).agg(
    {'avgImpressionRank': 'mean', 'adCost': 'sum', 'impression': 'sum', 'click': 'sum', 
     'directBuyConvCnt': 'sum', 'directBuyCostConvCnt': 'sum', 'indirectBuyConvCnt': 'sum', 'indirectBuyCostConvCnt': 'sum'}).reset_index() ###

summary_df_fin['buyConvCnt'] = summary_df_fin.directBuyConvCnt + summary_df_fin.indirectBuyConvCnt
summary_df_fin['buyCostConvCnt'] = summary_df_fin.directBuyCostConvCnt + summary_df_fin.indirectBuyCostConvCnt
summary_df_fin['cpa'] = np.where(summary_df_fin.buyConvCnt == 0, 0, summary_df_fin.adCost / summary_df_fin.buyConvCnt)
summary_df_fin['roas'] = np.where(summary_df_fin.adCost == 0, 0, summary_df_fin.buyCostConvCnt / summary_df_fin.adCost)

# 상품별 광고비 비중
campaign_slot = summary_df.groupby(['campaign', 'mediaDevice'])['adCost'].sum().unstack('mediaDevice')
campaign_slot = campaign_slot.fillna(0)

total_adCost = campaign_slot.sum(axis = 1)
total_adCost = total_adCost.values.reshape(len(total_adCost), 1) # dimension이 있어야 연산 가능

campaign_slot_ratio = campaign_slot / total_adCost
campaign_slot_ratio = campaign_slot_ratio.reset_index()
campaign_slot_ratio = campaign_slot_ratio[campaign_slot_ratio.daum_M.notnull()] # total_adCost = 0인 캠페인 제외

# 테이블과 형식 통일
campaign_slot_ratio.daum_M = round(campaign_slot_ratio.daum_M, 4)
campaign_slot_ratio.daum_P = round(campaign_slot_ratio.daum_P, 4)
campaign_slot_ratio.google_M = round(campaign_slot_ratio.google_M, 4)
campaign_slot_ratio.google_P = round(campaign_slot_ratio.google_P, 4)
campaign_slot_ratio.naver_M = round(campaign_slot_ratio.naver_M, 4)
campaign_slot_ratio.naver_P = round(campaign_slot_ratio.naver_P, 4)

## -------------------------------------------------------------------------------------------------------------------------------
### 2. 데이터마트 생성

In [ ]:
# 사이트별 캠페인 수 확인(KPI unique 깂이 5가지 이상인 사이트만 분석에 활용)
summary_df_kpi = summary_df_fin[summary_df_fin[kpi] != 0] # 지표가 0인 경우 제외
summary_df_kpi_tmp = summary_df_kpi[['site_id', kpi]].drop_duplicates()

summary_df_kpi_cnt = summary_df_kpi_tmp.groupby(['site_id']).agg({kpi: 'count'}).reset_index()
summary_df_kpi_cnt = summary_df_kpi_cnt.rename(columns = {kpi: 'cnt'})
summary_df_kpi_cnt = summary_df_kpi_cnt[summary_df_kpi_cnt.cnt >= 5]

summary_df_kpi2 = pd.merge(summary_df_kpi, summary_df_kpi_cnt, how = 'left', on = 'site_id')
summary_df_kpi2 = summary_df_kpi2[summary_df_kpi2.cnt.notnull()]

# 사이트별 KPI 그룹 번호 부여(특정 사이트만 분석에 반영되는 이슈 방지/CPA는 그룹 번호 반대로 생성)
group_df = []

if kpi =='cpa': 

    for col in summary_df_kpi2.site_name.unique():
        summary_df_kpi2_tmp = summary_df_kpi2.copy()
        summary_df_kpi2_tmp = summary_df_kpi2_tmp[summary_df_kpi2_tmp.site_name == col]
        summary_df_kpi2_tmp['cut'] = pd.qcut(summary_df_kpi2_tmp[kpi], q = 3, labels = [3, 2, 1], duplicates = 'drop')
        group_df.append(summary_df_kpi2_tmp)
        
else:

    for col in summary_df_kpi2.site_name.unique():
        summary_df_kpi2_tmp = summary_df_kpi2.copy()
        summary_df_kpi2_tmp = summary_df_kpi2_tmp[summary_df_kpi2_tmp.site_name == col]
        summary_df_kpi2_tmp['cut'] = pd.qcut(summary_df_kpi2_tmp[kpi], q = 3, labels = [1, 2, 3], duplicates = 'drop')
        group_df.append(summary_df_kpi2_tmp)

group_df = pd.concat(group_df)
group_df = group_df[['advertiser_id', 'advertiser_name', 'site_id', 'site_name', 'statYear', 'statMonth',
                     'adCost', 'click', 'buyConvCnt', 'buyCostConvCnt', 'cpa', 'roas', 'cut']] ###
group_df = group_df.astype({'statYear': str, 'statMonth': str})
group_df['campaign'] = group_df['advertiser_name'] + "_" + group_df['site_name'] + "_" + group_df['statYear'] + "_" + group_df['statMonth']

# 데이터마트 생성
kpi_df = pd.merge(group_df, campaign_slot_ratio, how = 'left', on = 'campaign')
kpi_df = kpi_df[kpi_df.daum_M.notnull()] # total_adCost = 0인 캠페인 제외

kpi_site = kpi_df[kpi_df.site_id == site_id] # 사이트 데이터셋
kpi_ad = kpi_df[kpi_df.advertiser_id == advertiser_id] # 광고주 데이터셋

## -------------------------------------------------------------------------------------------------------------------------------
### 3. 예산 분배 최적화

In [ ]:
# if (site_id is not None) and kpi_site.shape[0] >= 12: # 사이트 데이터 기준

# 데이터셋 준비
kpi_X = kpi_site[['cut']]
kpi_y = kpi_site[['daum_M', 'daum_P', 'google_M', 'google_P', 'naver_M', 'naver_P']]

# 최종 모형(사이트 모형 구축의 경우, 데이터 양이 부족하므로 데이터셋 분리 X)
kpi_reg  = LinearRegression().fit(kpi_X, kpi_y)
kpi_pred = kpi_reg.predict(kpi_X)

# print('uniform_average:', r2_score(kpi_y, kpi_pred, multioutput = 'uniform_average')) # r2 = ?
# print('variance_weighted:', r2_score(kpi_y, kpi_pred, multioutput = 'variance_weighted')) # r2 = ?

# 상위 그룹 기준 예산 분배
data = pd.DataFrame([3], index = [0])
kpi_output = kpi_reg.predict(data)

# 확률 형태로 변환
pd_limit = 0.001
kpi_output2 = np.where(kpi_output < pd_limit, 0, kpi_output) # 특정 비율보다 낮게 추정된 상품은 0으로 변환
kpi_output_sum = np.sum(kpi_output2, axis = 1, keepdims = True)

kpi_output2 = pd.DataFrame(kpi_output2)
kpi_output_sum = np.where(kpi_output_sum == 0, 0.00001, kpi_output_sum) # 합을 0이 아닌 아주 작은 값으로 변환

kpi_pred_pb = np.where(kpi_output2 == 0, 0, kpi_output2 / kpi_output_sum)
kpi_pred_pb = pd.DataFrame(kpi_pred_pb, columns = kpi_y.columns)

kpi_pred_pb = kpi_pred_pb.T.reset_index()
kpi_pred_pb.columns = ['mediaDevice', 'ad_proportion']

# 평균 효율(CPC/전환율/객단가)
kpi_site2 = kpi_site[kpi_site.cut == 3]
kpi_site2 = kpi_site2[['campaign', 'cut']]

summary_df2 = pd.merge(summary_df, kpi_site2, how = 'left', on = ['campaign'])
summary_df2 = summary_df2[summary_df2.cut.notnull()]

summary_df2['buyConvCnt'] = summary_df2.directBuyConvCnt + summary_df2.indirectBuyConvCnt
summary_df2['buyCostConvCnt'] = summary_df2.directBuyCostConvCnt + summary_df2.indirectBuyCostConvCnt

kpi_summary = summary_df2.groupby(['mediaDevice']).agg({
    'adCost': 'sum', 'click': 'sum', 'buyConvCnt': 'sum', 'buyCostConvCnt': 'sum'}).reset_index()

kpi_summary['cpc'] = np.where(kpi_summary.click == 0, 0, kpi_summary.adCost / kpi_summary.click)
kpi_summary['cpa'] = np.where(kpi_summary.buyConvCnt == 0, 0, kpi_summary.adCost / kpi_summary.buyConvCnt)
kpi_summary['cvr'] = np.where(kpi_summary.click == 0, 0, kpi_summary.buyConvCnt / kpi_summary.click)
kpi_summary['ct'] = np.where(kpi_summary.buyConvCnt == 0, 0, kpi_summary.buyCostConvCnt / kpi_summary.buyConvCnt) # 객단가

# 미디어믹스
result_fin = pd.merge(kpi_pred_pb, kpi_summary[['mediaDevice', 'cpc', 'cvr', 'ct']], how = 'left', on = 'mediaDevice')

result_fin['ad_price'] = round(result_fin.ad_proportion * budget, -5)
result_fin['cpc'] = np.where(result_fin.cpc.isnull(), 0, round(result_fin.cpc, -1))
result_fin['cvr'] = np.where(result_fin.cvr.isnull(), 0, round(result_fin.cvr, 3))
result_fin['ct'] = np.where(result_fin.ct.isnull(), 0, round(result_fin.ct, 0))

result_fin

,mediaDevice,ad_proportion,cpc,cvr,ct,ad_price
0,daum_M,0.000000,0.0,0.000,0.0,0.0
1,daum_P,0.000000,0.0,0.000,0.0,0.0
2,google_M,0.213461,570.0,0.026,87016.0,21300000.0
3,google_P,0.323870,850.0,0.077,140631.0,32400000.0
4,naver_M,0.241404,580.0,0.000,74029.0,24100000.0
5,naver_P,0.221265,780.0,0.086,154636.0,22100000.0
